In [266]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtuples as tt

from pycox.models import CoxPH
from pycox.models import DeepHitSingle
from pycox.evaluation import EvalSurv
import warnings
from sklearn.model_selection import KFold
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest

np.random.seed(12)
_ = torch.manual_seed(12)

In [267]:
class TransformerSurvival(nn.Module):
    def __init__(self, in_features, d_model=32, out_features=1, 
                 nhead=4, num_layers=2, dropout=0.1, batch_norm=True, output_bias=False):
        super().__init__()
        
        # Input projection - 将输入特征投影到Transformer的维度
        self.input_proj = nn.Linear(in_features, d_model)
        
        # Positional encoding - 位置编码
        self.pos_encoding = nn.Parameter(torch.randn(1, 1, d_model))
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=d_model * 4,
            dropout=dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)
        
        # Batch norm
        self.bn = nn.BatchNorm1d(d_model) if batch_norm else nn.Identity()
        
        # 直接输出层 - 从Transformer维度映射到输出维度
        self.output_layer = nn.Linear(d_model, out_features, bias=output_bias)
        
        self.dropout = nn.Dropout(dropout)
        self.d_model = d_model

    def forward(self, x):
        # Input processing
        x = self.input_proj(x)  # [batch_size, in_features] -> [batch_size, d_model]
        x = self.bn(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        # Transformer processing
        # 添加序列维度并加上位置编码
        x = x.unsqueeze(1)  # [batch_size, d_model] -> [batch_size, 1, d_model]
        x = x + self.pos_encoding
        
        # Transformer处理
        x = self.transformer(x)  # [batch_size, 1, d_model]
        
        # 移除序列维度并直接输出
        x = x.squeeze(1)  # [batch_size, 1, d_model] -> [batch_size, d_model]
        
        # 直接映射到输出
        return self.output_layer(x)


# 简洁的创建函数
def create_transformer_network(in_features, d_model=32, out_features=1, 
                              batch_norm=True, dropout=0.1, output_bias=False,
                              nhead=4, num_layers=2):
    """创建简化的Transformer生存网络（无中间输出层）"""
    return TransformerSurvival(
        in_features=in_features,
        d_model=d_model,
        out_features=out_features,
        nhead=nhead,
        num_layers=num_layers,
        dropout=dropout,
        batch_norm=batch_norm,
        output_bias=output_bias
    )

In [268]:
import pandas as pd
# 读取数据
df = pd.read_csv('C:/Users/Administrator/Desktop/pycox-master/examples/TranSurv/2025.08.28/others/high data/1-Transbig_screening.csv')
print(df.head())
n_cols = df.shape[1]
# 前 n_cols-2 个是 x1, x2, ..., xk
x_cols = [f"x{i+1}" for i in range(n_cols - 2)]
df.columns = ["duration", "event"] + x_cols  

   time  status  X200760_s_at  X203485_at  X209368_at  X204267_x_at  \
0   723       1     -0.106307   -0.087483    0.011426     -0.005646   
1  6591       0     -0.054147    0.044850   -0.050012      0.064526   
2   524       1     -0.040328   -0.012857   -0.119295     -0.051362   
3  6255       1     -0.058828    0.034322   -0.055834      0.059381   
4  3822       1     -0.053449   -0.052275    0.044296      0.110822   

   X212070_at  X200761_s_at  X208968_s_at  X203162_s_at  ...  X200708_at  \
0    0.045452     -0.035139      0.083255      0.039176  ...    0.034016   
1   -0.049141     -0.055614      0.008711      0.061409  ...   -0.064124   
2    0.045432     -0.029518     -0.044925      0.004279  ...   -0.037367   
3   -0.001969     -0.043834     -0.007116      0.056420  ...    0.019109   
4    0.095221     -0.066307      0.094748      0.046209  ...    0.039998   

   X201291_s_at  X214343_s_at  X202240_at  X210171_s_at  X204023_at  \
0     -0.041813     -0.040715    0.026708    

In [269]:
cols_standardize = x_cols

standardize = [([col], StandardScaler()) for col in cols_standardize]

x_mapper = DataFrameMapper(standardize)
print(df.head())
print(df.shape)
print("删失率:", 1 - df["event"].mean())

   duration  event        x1        x2        x3        x4        x5  \
0       723      1 -0.106307 -0.087483  0.011426 -0.005646  0.045452   
1      6591      0 -0.054147  0.044850 -0.050012  0.064526 -0.049141   
2       524      1 -0.040328 -0.012857 -0.119295 -0.051362  0.045432   
3      6255      1 -0.058828  0.034322 -0.055834  0.059381 -0.001969   
4      3822      1 -0.053449 -0.052275  0.044296  0.110822  0.095221   

         x6        x7        x8  ...       x28       x29       x30       x31  \
0 -0.035139  0.083255  0.039176  ...  0.034016 -0.041813 -0.040715  0.026708   
1 -0.055614  0.008711  0.061409  ... -0.064124  0.077107 -0.020908  0.085010   
2 -0.029518 -0.044925  0.004279  ... -0.037367  0.054396 -0.029566  0.044774   
3 -0.043834 -0.007116  0.056420  ...  0.019109  0.027086 -0.005127  0.078900   
4 -0.066307  0.094748  0.046209  ...  0.039998 -0.047335  0.152569  0.067730   

        x32       x33       x34       x35       x36       x37  
0  0.091858  0.019393 

In [270]:
# 设置重复次数和折数
n_repeats = 20
n_folds = 5
total_runs = n_repeats * n_folds
batch_size = 32
epochs = 200

# 初始化结果存储
cindex = np.zeros((total_runs, 5))
ibs = np.zeros((total_runs, 5))

print(f"开始进行 {n_repeats} 次重复的 {n_folds} 折交叉验证，总共 {total_runs} 次运行...")
j=-1
for repeat in range(n_repeats):
    if repeat%10==0:
        print(f"第 {repeat + 1}/{n_repeats} 次重复...")
    
    # 每次重复时随机打乱数据
    df_shuffled = df.sample(frac=1, random_state=repeat).reset_index(drop=True)
    
    # 初始化KFold
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=repeat)
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(df_shuffled)):
        # 划分训练集和测试集
        j=j+1
        df_train = df_shuffled.iloc[train_idx]
        df_test = df_shuffled.iloc[test_idx]
        
        # 从训练集中划分验证集
        df_val = df_train.sample(frac=0.2, random_state=repeat)
        df_train = df_train.drop(df_val.index)
        
        # 数据预处理
        x_train = x_mapper.fit_transform(df_train).astype('float32')
        x_val = x_mapper.transform(df_val).astype('float32')
        x_test = x_mapper.transform(df_test).astype('float32')
        
        # 数据预处理 - 传统机器学习模型（使用所有训练数据）
        x_train_ml = x_mapper.fit_transform(df_train).astype('float32')
        x_test_ml = x_mapper.transform(df_test).astype('float32')
        
        get_target = lambda df: (df['duration'].values, df['event'].values)
        y_train = get_target(df_train)
        y_val = get_target(df_val)
        durations_test, events_test = get_target(df_test)
        
        ## 专门为DeepHit设计，转换成离散时间
        num_durations = 10
        labtrans = DeepHitSingle.label_transform(num_durations)
        y_train2 = labtrans.fit_transform(*get_target(df_train))
        y_val2 = labtrans.transform(*get_target(df_val))
        train2 = (x_train, y_train2)
        val2 = (x_val, y_val2)
        
        val = (x_val, y_val)
        # 准备生存分析格式（用于Cox和RSF）
        def prepare_survival_data(df):
            return np.array([(row['event'], row['duration']) for _, row in df.iterrows()], 
                          dtype=[('event', '?'), ('time', '<f8')])
        
        y_train_surv = prepare_survival_data(df_train)
        y_test_surv = prepare_survival_data(df_test)
        
        callbacks1 = [tt.callbacks.EarlyStopping()]
        callbacks2 = [tt.callbacks.EarlyStopping()]
        callbacks3 = [tt.callbacks.EarlyStopping()]
        
        # 创建和训练模型
        net1 = create_transformer_network(
            in_features= x_train.shape[1],
            d_model=64,
            out_features=1,
            nhead=2,
            num_layers=1,
            dropout=0.1,
            batch_norm=True,
            output_bias=False
        )
        model1 = CoxPH(net1, tt.optim.Adam)
        model1.optimizer.set_lr(0.01)

        ## model 2
        net2 = tt.practical.MLPVanilla(in_features = x_train.shape[1], num_nodes = [32, 32], out_features = 1, batch_norm = True,
                                      dropout = 0.1,  output_bias = False)
        model2 = CoxPH(net2, tt.optim.Adam)
        model2.optimizer.set_lr(0.01)
        
        ## model 3
        net3 = tt.practical.MLPVanilla(in_features = x_train.shape[1], num_nodes = [32, 32], out_features = labtrans.out_features, 
                                      batch_norm = True, dropout = 0.1)
        model3 = DeepHitSingle(net3, tt.optim.Adam, alpha=0.2, sigma=0.1, duration_index=labtrans.cuts)
        model3.optimizer.set_lr(0.01)
        
        # 模型4: Cox比例风险模型
        cox_model = CoxPHSurvivalAnalysis()
        
        # 模型5: 随机生存森林
        rsf_model = RandomSurvivalForest(
            n_estimators= 50,
            min_samples_split=10,
            min_samples_leaf=15,
            max_features="sqrt",
            n_jobs=-1,
            random_state=repeat
        )
        
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=FutureWarning, module="torchtuples")
            log1 = model1.fit(x_train, y_train, batch_size, epochs, callbacks1, False,
                          val_data=val, val_batch_size=batch_size)
            log2 = model2.fit(x_train, y_train, batch_size, epochs, callbacks2, False,
                          val_data=val, val_batch_size=batch_size)
            log3 = model3.fit(x_train, y_train2, batch_size, epochs, callbacks3, False, 
                              val_data=val2, val_batch_size=batch_size)
            # 训练传统机器学习模型
            cox_model.fit(x_train_ml, y_train_surv)
            rsf_model.fit(x_train_ml, y_train_surv)
            
        # 计算基线风险并预测
        time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
        ha1 = model1.compute_baseline_hazards()
        surv1 = model1.predict_surv_df(x_test)
        ha2 = model2.compute_baseline_hazards()
        surv2 = model2.predict_surv_df(x_test)
        
        ## DeepHit预测
        surv3 = model3.interpolate(num_durations).predict_surv_df(x_test)
        
        # Cox模型预测 - 插值到统一的时间网格
        cox_surv_fn = cox_model.predict_survival_function(x_test_ml)
        cox_max_time = min(fn.x[-1] for fn in cox_surv_fn)   # 每个 StepFunction 最后一个点
        time_grid1 = np.linspace(durations_test.min(), cox_max_time, 100)
        cox_surv = np.zeros((len(x_test_ml), len(time_grid1)))
        for i, fn in enumerate(cox_surv_fn):
            cox_surv[i] = np.array([fn(t) for t in time_grid1])
        surv4 = pd.DataFrame(cox_surv.T, index=time_grid1)
        
        # RSF模型预测 - 插值到统一的时间网格
        rsf_surv_fn = rsf_model.predict_survival_function(x_test_ml)
        rsf_max_time = min(fn.x[-1] for fn in rsf_surv_fn)   # 每个 StepFunction 最后一个点
        time_grid2 = np.linspace(durations_test.min(), rsf_max_time, 100)
        rsf_surv = np.zeros((len(x_test_ml), len(time_grid2)))
        for i, fn in enumerate(rsf_surv_fn):
            rsf_surv[i] = np.array([fn(t) for t in time_grid2])
        surv5 = pd.DataFrame(rsf_surv.T, index=time_grid2)

        # 评估模型
        ev1 = EvalSurv(surv1, durations_test, events_test, censor_surv='km')
        cindex[j,4] = ev1.concordance_td()
        ev2 = EvalSurv(surv2, durations_test, events_test, censor_surv='km')
        cindex[j,3] = ev2.concordance_td()
        ev3 = EvalSurv(surv3, durations_test, events_test, censor_surv='km')
        cindex[j,2] = ev3.concordance_td('antolini')
        ev4 = EvalSurv(surv4, durations_test, events_test, censor_surv='km')
        cindex[j,0] = ev4.concordance_td()
        ev5 = EvalSurv(surv5, durations_test, events_test, censor_surv='km')
        cindex[j,1] = ev5.concordance_td()
        
        ibs[j,4] = ev1.integrated_brier_score(time_grid)
        ibs[j,3] = ev2.integrated_brier_score(time_grid)
        ibs[j,2] = ev3.integrated_brier_score(time_grid)
        ibs[j,0] = ev4.integrated_brier_score(time_grid)
        ibs[j,1] = ev5.integrated_brier_score(time_grid)
        if j%50==0:
            print(f"  折 {fold + 1}/{n_folds} - C-index1: {cindex[j,0]:.4f}, C-index2: {cindex[j,1]:.4f},  C-index3: {cindex[j,2]:.4f}, C-index4: {cindex[j,3]:.4f}, C-index5: {cindex[j,4]:.4f}")
            print(f"  折 {fold + 1}/{n_folds} - IBS1: {ibs[j,0]:.4f}, IBS2: {ibs[j,1]:.4f},  IBS3: {ibs[j,2]:.4f},  IBS4: {ibs[j,3]:.4f},  IBS5: {ibs[j,4]:.4f}")

# 计算平均指标
mean_cindex = np.mean(cindex, axis=0)
mean_ibs = np.mean(ibs, axis=0)
std_cindex = np.std(cindex, axis=0)
std_ibs = np.std(ibs, axis=0)

print(f"\n最终结果 (基于 {total_runs} 次运行):")
print(f"模型1- Concordance Index: {mean_cindex[0]:.3f} ({std_cindex[0]:.3f})")
print(f"模型2- Concordance Index: {mean_cindex[1]:.3f} ({std_cindex[1]:.3f})")
print(f"模型3- Concordance Index: {mean_cindex[2]:.3f} ({std_cindex[2]:.3f})")
print(f"模型4- Concordance Index: {mean_cindex[3]:.3f} ({std_cindex[3]:.3f})")
print(f"模型5- Concordance Index: {mean_cindex[4]:.3f} ({std_cindex[4]:.3f})")
print(f"模型1- Integrated Brier Score: {mean_ibs[0]:.3f} ({std_ibs[0]:.3f})")
print(f"模型2- Integrated Brier Score: {mean_ibs[1]:.3f} ({std_ibs[1]:.3f})")
print(f"模型3- Integrated Brier Score: {mean_ibs[2]:.3f} ({std_ibs[2]:.3f})")
print(f"模型4- Integrated Brier Score: {mean_ibs[3]:.3f} ({std_ibs[3]:.3f})")
print(f"模型5- Integrated Brier Score: {mean_ibs[4]:.3f} ({std_ibs[4]:.3f})")

开始进行 20 次重复的 5 折交叉验证，总共 100 次运行...
第 1/20 次重复...
  折 1/5 - C-index1: 0.8605, C-index2: 0.6802,  C-index3: 0.8256, C-index4: 0.8314, C-index5: 0.7733
  折 1/5 - IBS1: 0.1107, IBS2: 0.1840,  IBS3: 0.2057,  IBS4: 0.1661,  IBS5: 0.2665
第 11/20 次重复...
  折 1/5 - C-index1: 0.4821, C-index2: 0.5250,  C-index3: 0.5357, C-index4: 0.5607, C-index5: 0.5143
  折 1/5 - IBS1: 0.3129, IBS2: 0.1679,  IBS3: 0.1991,  IBS4: 0.2338,  IBS5: 0.2776

最终结果 (基于 100 次运行):
模型1- Concordance Index: 0.747 (0.081)
模型2- Concordance Index: 0.724 (0.071)
模型3- Concordance Index: 0.731 (0.072)
模型4- Concordance Index: 0.758 (0.072)
模型5- Concordance Index: 0.778 (0.068)
模型1- Integrated Brier Score: 0.185 (0.058)
模型2- Integrated Brier Score: 0.167 (0.021)
模型3- Integrated Brier Score: 0.205 (0.039)
模型4- Integrated Brier Score: 0.173 (0.046)
模型5- Integrated Brier Score: 0.170 (0.042)
